In [10]:
import os
import string
import re

import gensim
from gensim.models import word2vec

import nltk
from nltk import corpus
from nltk.tokenize import TweetTokenizer

In [11]:
folder = 'questions'
files = [file for file in os.listdir(folder) if file.endswith('.txt')]

print(len(files))

45571


In [12]:
raw_questions = []

for file in files:
    with open(os.path.join(folder, file)) as f:
        raw_questions.append(f.read())

Let's save raw, unprocessed questions:

In [13]:
with open('raw_questions.txt', 'w') as f:
    f.write('\n'.join(raw_questions))

In [14]:
raw_questions[0]

'Кто знает сайты, где можно качать бесплатные mp3?'

# Processing

## Уберём пунктуацию и стоп-слова

1\. Пунктуация

In [15]:
print(string.punctuation)
punctuation = '"#$%&\'()*+/:,-.?!;<=>@[\]^_`{|}~'

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [16]:
# questions = [''.join([ch for ch in q if ch not in punctuation]).lower()  for q in raw_questions]
# for q in questions[:5]: print(q)

2\. Уберём из предложений также стоп-слова из `nltk.corpus.stopwords.words('russian')`.

In [17]:
stopwords = list(set(corpus.stopwords.words('russian')))

print(len(stopwords))
print(str(stopwords[:15]))

151
['во', 'том', 'того', 'со', 'нас', 'вас', 'мне', 'то', 'для', 'моя', 'есть', 'про', 'зачем', 'другой', 'чтобы']


In [18]:
def raw_question_to_clean(raw_question, stopwords = stopwords):
    """
    raw_question: str
    stopwords: list of str
    
    returns: str (question without stopwords and punctuation)
    e.g:
        raw_question = 'С чем будет связана моя будущая работа? кем лучше стать? 25.09.1999'
        returns 'связана будущая работа кем стать 25 09 1999'
    """
    
    letters_only = re.sub('[^a-zA-Zа-яА-я0-9]', ' ', raw_question) 
    
    words = letters_only.lower().split()                                                  
    meaningful_words = [w for w in words if not w in stopwords]   

    return(' '.join( meaningful_words ))   

In [19]:
questions = [raw_question_to_clean(q) for q in raw_questions]

Сравним почищенные (`questions`) и исходные, необработанные вопросы (`raw_questions`):

In [20]:
for q, rq in zip(questions[:5], raw_questions[:5]): print('before: {}\nafter:  {}\n'.format(rq, q))

before: Кто знает сайты, где можно качать бесплатные mp3?
after:  знает сайты качать бесплатные mp3

before: почему так часто встречаються люди хамы?
after:  почему часто встречаються люди хамы

before: Почему ногти на больших пальцах рук растут медленнее, чем на остальных (указательном, среднем, безымянном и мизинце)?
after:  почему ногти больших пальцах рук растут медленнее остальных указательном среднем безымянном мизинце

before: у кого есть слайд-гитара?
after:  кого слайд гитара

before: как удалить свой вопрос?
after:  удалить свой вопрос



Очевидно, что просто убирать пунктуацию не очень эффективно. Да и без стоп-слов вопросы с Ответов.Mailru кажутся полным бредом. 

In [21]:
# nltk.data.load('tokenizers/punkt/russian.pickle')

------

## Попробуем `TweetTokenizer()`

In [22]:
def tokenize_question(raw_question, tokenizer = TweetTokenizer(), stopwords = None):
    """
    raw_question: str
    tokenizer: tokenizer, default: TweetTokenizer()
    stopwords: list, default: None
    
    returns: list of str (tokenized question)
    e.g.:
        raw_question = Иисус это Вселенная?
        returns ['иисус', 'это', 'вселенная', '?']     
    """
    words = tokenizer.tokenize(raw_question.lower())
                     
    if stopwords:
        words = [w for w in words if not w in stopwords]   

    return words

In [23]:
tokens = [tokenize_question(q) for q in raw_questions]

In [24]:
print('sentenses: ', len(tokens))
print('words: ', len([token for sent in tokens for token in sent]))
print('unique words: ', len(set([token for sent in tokens for token in sent])))

sentenses:  45571
words:  524480
unique words:  69346


In [25]:
for t in tokens[:5]: print(str(t))

['кто', 'знает', 'сайты', ',', 'где', 'можно', 'качать', 'бесплатные', 'mp3', '?']
['почему', 'так', 'часто', 'встречаються', 'люди', 'хамы', '?']
['почему', 'ногти', 'на', 'больших', 'пальцах', 'рук', 'растут', 'медленнее', ',', 'чем', 'на', 'остальных', '(', 'указательном', ',', 'среднем', ',', 'безымянном', 'и', 'мизинце', ')', '?']
['у', 'кого', 'есть', 'слайд-гитара', '?']
['как', 'удалить', 'свой', 'вопрос', '?']


Окей, выглядит неплохо, но там наверняка много мусора.

* Добавим `reduce_len = True` (вместо 53893 уникальных слов мы получим 53881): 

In [26]:
tokens_reduce_len = [tokenize_question(q, TweetTokenizer(reduce_len = True)) 
                     for q in raw_questions]

In [27]:
print('sentenses: ', len(tokens_reduce_len))
print('words: ', len([token for sent in tokens_reduce_len for token in sent]))
print('unique words: ', len(set([token for sent in tokens_reduce_len for token in sent])))

sentenses:  45571
words:  524480
unique words:  69330


In [28]:
for t in list(set([token for sent in tokens for token in sent])
              .difference(set([token for sent in tokens_reduce_len for token in sent])))[:10]: print(t)

способствуетtttttt
70000
2024561111
0xc0000005
20000-25000
150000тдол
10000р
zzzz
40000р
0,000011


* Добавим `strip_handles = True` к `reduce_len = True` (вместо 53881 уникальных слов мы получим 53878). 

(Потеряются слова `{'@zakladkis', '@kiska', '@instagram'}`. На мой взгляд, особой важности в них не было.)

In [29]:
tokens_reduce_strip = [tokenize_question(q, TweetTokenizer(reduce_len    = True, 
                                                           strip_handles = True)) 
                       for q in raw_questions]

In [30]:
print('sentenses: ', len(tokens_reduce_strip))
print('words: ', len([token for sent in tokens_reduce_strip for token in sent]))
print('unique words: ', len(set([token for sent in tokens_reduce_strip for token in sent])))

sentenses:  45571
words:  524474
unique words:  69325


In [31]:
for t in (set([token for sent in tokens_reduce_len for token in sent])
          .difference(set([token for sent in tokens_reduce_strip for token in sent]))): print(t)

@instagram
@echo
@zakladkis
@kiska
@medi


------

In [32]:
def is_word(token):
    return not re.search(r'^[\w\d\-]+$', token) is None

In [33]:
def is_okay(token):
    token_is_word  = is_word(token)
    token_is_punct = token in [',', '-', ':']
    
    return token_is_word or token_is_punct

In [34]:
# token_list = tokens_reduce_strip[67]
# print(str(token_list))

# for token in token_list: print(token, '\t\t\t', re.search(r'^[\w\d\-]+$' ,token) is None)
# for token in token_list: print(token, '\t\t\t', is_word(token))
# for token in token_list: print(token, '\t\t', is_okay(token))

In [35]:
questions_tokenized = []

for token_list in tokens_reduce_strip:
    token_list_ok = [t for t in token_list if is_okay(t)]
    question = ' '.join(token_list_ok)
    questions_tokenized.append(question)

In [36]:
for q in questions_tokenized[:5]: print(str(q))

кто знает сайты , где можно качать бесплатные mp3
почему так часто встречаються люди хамы
почему ногти на больших пальцах рук растут медленнее , чем на остальных указательном , среднем , безымянном и мизинце
у кого есть слайд-гитара
как удалить свой вопрос


------

In [45]:
def remove_space_before(string):
    """
    string: str
    returns: str (without spaces before commas and colons)
    e.g.:
        string = 'дять , а 2 провода которые вышли от транса куда уйдет'
        returns 'дять, а 2 провода которые вышли от транса куда уйдет'
    """
    s = re.sub(' ,', ',', string)
    s = re.sub(' :', ':', s)
    return  s

------

Тут марковчейн для развлечения, пока вопросы парсятся.

In [37]:
import markovify
import pymarkovchain

# mv = markovify.Text('\n'.join(questions), state_size = 2)
# for _ in range(50): print(mv.make_sentence(), '\n')

In [39]:
mc = pymarkovchain.MarkovChain()
mc.generateDatabase('\n'.join(questions_tokenized))

In [46]:
for _ in range(50): print('- ' + (remove_space_before(mc.generateString())) + '?')

- на нас были от сотрудников в гит?
- что делать?
- лучший?
- людии, помогите вспомнить название британской сказки автор - женщина, с кем обсуждать биаса, дорамы и т д?
- помогите пожалуйста?
- как быть гуманным, если у меня гта 5 возможно как-то через торрент отправить папку, создал торрент файл но он про него?
- ларгус установка бк?
- как долго хватит gtx 1080 vga мониторы, не дотягиваюсь до звонка: что бы было по жоще, подскажите что даёт жизнь?
- денег на фильтры?
- стоит ли нести в юстировку 24-70 l ii, если ждать бесполезно?
- верно говорят, она видела уверенного и физизески развитого?
- сааб 9-5 2000 года, депрессия все время, что за культ жоп и кача в инстаграм вставить цифры в в москве вкусно, быстро, но тут так мало популярной французской музыки, серфинга по инету вообще реальная вещь?
- где можно скачать?
- собеседник меня не стало что делать нам?
- почему мужчины после 50 такие потрепанный и помятые почему?
- как спорт влияют на кашель?
- хочу подать в суд уголовное дело ес